In [21]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

import pickle
from sklearn.externals import joblib
import os
import re

In [22]:
# df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [23]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);


In [24]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [25]:
X={}
X_raw={}
for sku in list(df_dict.keys()):
    X_raw[sku]=df_dict[sku]
    X_raw[sku]=X_raw[sku].drop(['id','DayOfWeek','High_Var',"Luxury",'Customers','Sales'],axis=1)

In [26]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X_raw[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)
        

In [27]:
for sku in list(df_dict.keys()):
    X_raw[sku]["Mov_avg"]=df_dict[sku]['Sales'].rolling( window=7).mean().shift(1)

In [28]:
len(df)

9420

In [29]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X_raw[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [30]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [31]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X[sku])
    X[sku] = std[sku].transform(X[sku])


In [32]:
cv=2

RF=RandomForestRegressor( n_estimators=1000,max_depth=10,max_features=14);
# use a full grid over all parameters


rf_models=[];
for sku in list(df_dict.keys()):
    rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0) )
#


In [33]:

GB=ensemble.GradientBoostingRegressor(n_estimators= 1000,max_depth= 8, max_features=14)#,learning_rate=0.01,loss='ls');
gb_models=[];
for sku in list(df_dict.keys()):
    gb_models.append(cross_val_score(GB,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
#    


In [34]:
XG=xgb.XGBRegressor(n_estimators= 900,learning_rate =0.01, max_depth=6,gamma=2  )
xg_models=[]
for sku in list(df_dict.keys()):
    xg_models.append(cross_val_score(XG,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [35]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X[1].shape[1], activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    #mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))
    mlp.add(Dense(1))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=400,
                               batch_size=20,
                               verbose=0)

In [36]:
mlp_models=[]
for sku in list(df_dict.keys()):
    mlp_models.append(cross_val_score(MLP,X[sku],y[sku],cv=cv).mean().round(0)  )

In [37]:
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae.index = mae.index + 1
mae.loc['mean'] = mae.mean()
mae['range']=mae.max(axis=1)-mae.min(axis=1)
mae

,MLP,RF,GB,XG,Best,range
1,420.0,360.0,380.0,361.0,RF,60.0
2,456.0,384.0,410.0,386.0,RF,72.0
3,724.0,584.0,624.0,597.0,RF,140.0
4,785.0,747.0,801.0,793.0,RF,54.0
5,417.0,369.0,386.0,384.0,RF,48.0
6,651.0,552.0,574.0,601.0,RF,99.0
7,790.0,726.0,751.0,742.0,RF,64.0
8,681.0,485.0,485.0,506.0,RF,196.0
9,584.0,655.0,698.0,727.0,MLP,143.0
10,448.0,385.0,411.0,401.0,RF,63.0


In [38]:
#Need lots of rank and counts of how many times models came in 1st  vs 2nd.
def rank_model(df,rank):
    coln = 'Best' + str(rank) 
    sortID = np.argpartition(df[['MLP','RF','GB','XG']].values,rank,axis=1)[:,rank]
    df[coln] = df.columns[sortID]
    

In [39]:
rank_model(mae,0)
rank_model(mae,1)
mae

,MLP,RF,GB,XG,Best,range,Best0,Best1
1,420.0,360.0,380.0,361.0,RF,60.0,RF,XG
2,456.0,384.0,410.0,386.0,RF,72.0,RF,XG
3,724.0,584.0,624.0,597.0,RF,140.0,RF,XG
4,785.0,747.0,801.0,793.0,RF,54.0,RF,MLP
5,417.0,369.0,386.0,384.0,RF,48.0,RF,XG
6,651.0,552.0,574.0,601.0,RF,99.0,RF,GB
7,790.0,726.0,751.0,742.0,RF,64.0,RF,XG
8,681.0,485.0,485.0,506.0,RF,196.0,RF,GB
9,584.0,655.0,698.0,727.0,MLP,143.0,MLP,RF
10,448.0,385.0,411.0,401.0,RF,63.0,RF,XG


In [40]:
def RF_final(X, y):
    """
    Returns final predictions and final fitted version of RF
    """
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return (pred, tree)

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return (pred, model)

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return (pred, model)

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return (pred, model)

In [41]:
#NOTE, because we are retraining models here, a bad initiatlization could lead to different results. 

def best_model(scores,X,y):
    d={}
    for sku in list(df_dict.keys()):
        
        if scores.loc[sku]["Best"]=="RF":     
            d[sku]=RF_final(X[sku],y[sku])
                    
        elif scores.loc[sku]["Best"]=="MLP":
            d[sku]=MLP_final(X[sku],y[sku])                 
            
        elif scores.loc[sku]["Best"]=="GB":
            d[sku]=GB_final(X[sku],y[sku])
            
        elif scores.loc[sku]["Best"]=="XG":
            d[sku]=XG_final(X[sku],y[sku])
      
    return d     

In [42]:
bm_dict=best_model(mae,X,y)
# bm_dict

In [43]:
#MAE of Best Model forecasting over all 935 days.  NN do very poorly here!
mae2=[]
for sku in list(df_dict.keys()):
    mae2.append( mean_absolute_error(y[sku], bm_dict[sku][0])   )
mae2=pd.DataFrame(mae2)
mae2.index = mae2.index + 1
mae2.round(1)

,0
1,191.9
2,215.9
3,304.6
4,361.3
5,197.2
6,189.9
7,352.6
8,201.5
9,391.7
10,198.8


In [44]:
sku_pred=pd.concat([ bm_dict[1][0],bm_dict[2][0],bm_dict[3][0],bm_dict[4][0],bm_dict[5][0],bm_dict[6][0],bm_dict[7][0],bm_dict[8][0],bm_dict[9][0],bm_dict[10][0],
],axis=1) 
sku_pred['Total'] = sku_pred.sum(axis=1)
sku_pred.columns = ['1','2','3','4','5','6','7','8','9','10','Total']
# sku_pred.to_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred.to_csv('./prediction_skus.csv')

#Each column in this dataframe is the result of 1 of the 4 models
sku_pred

,1,2,3,4,5,6,7,8,9,10,Total
0,5717.818209,6301.653071,9865.725643,10153.291251,5602.367520,7176.216977,10341.561739,6505.311482,6286.657227,5714.657222,73665.260340
1,5510.298604,6512.066582,8556.524297,9330.420316,5765.830248,6921.244257,8090.852354,5673.728950,5216.300293,5539.029505,67116.295407
2,5079.047790,5460.737801,7735.243498,9324.782816,4944.436392,6314.656802,8066.825115,6234.547054,5395.656738,5616.178584,64172.112592
3,4855.066339,4907.510546,8600.125698,9873.201129,5069.712080,6927.614117,8099.700803,5206.362083,5717.865723,5606.811352,64863.969870
4,4848.817431,2609.258951,4299.631053,9935.107310,1835.048437,3765.943200,4538.281949,2510.754229,4943.070801,4541.323872,43827.237233
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.162793,0.000000,0.162793
6,4364.450817,4004.262165,5161.439033,7940.504973,3824.709259,5972.879755,6844.887051,4286.980709,4499.971680,4817.428118,51717.513560
7,4031.301953,3947.826898,5158.895142,7462.205307,3727.959016,5616.944218,6807.330886,4192.341320,4182.101074,4569.376346,49696.282161
8,3944.842050,4649.660839,5124.796593,7195.689326,3997.489338,5247.771443,6372.850673,3895.344289,3797.039307,4551.669918,48777.153776
9,3894.189293,3881.791308,5029.780984,7370.564596,3713.124132,5554.787497,6780.353349,4827.428066,4405.817383,4520.857199,49978.693807


In [45]:
# The rest of this notebook is just a reference and aid to allow collaborators to... 
# - save models
# - retrive and apply them without retraining. 

def save_model(model, filename):
    """
    Saves the model as a file. 
    - Keras models: saved as .json and .h5 files
    - scikit-learn models: saved as .pickle files
    """
    
    if type(model) is KerasRegressor:
        
        model_json = model.model.to_json()
        with open('{}.json'.format(filename), "w") as json_file:
            json_file.write(model_json)
        model.model.save_weights('{}.h5'.format(filename))        
    else:
        pickle_out = open('{}.pickle'.format(filename), "wb")
        pickle.dump(model, pickle_out)
        pickle_out.close()

    
# Save each SKU's best model to the local directory
for sku in list(df_dict.keys()):

    filename="SKU{}_best_model".format(sku)
    model = bm_dict[sku][1]

    save_model(model, filename)



In [46]:
# Load pickled models
# loaded_model = pickle.load(open('SKU1_best_model.pickle', 'rb'))
# print(type(loaded_model) is RandomForestRegressor)

def load_pickled_model(path_to_model):
    loaded_model = pickle.load(open(path_to_model, 'rb'))
    
    return loaded_model

def load_keras_model(path_to_json, path_to_weights_h5):
    
    # load json and create model
    json_file = open(path_to_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights(path_to_weights_h5)
    
    # wrap Sequential obj as a KerasRegressor to allow scikit-learn compatibility
    wrapped_loaded_model = KerasRegressor(build_fn=create_MLP, epochs=400, batch_size=20, verbose=0)
    
    return wrapped_loaded_model

def load_model(filename):
    """
    Loads a given file as a model
    - .pickle files: loaded as scikit-learn models
    - .json and .h5 files: loaded together to create Keras models
    """
    
    loaded_model = None
    
    try:
        loaded_model = load_pickled_model("{}.pickle".format(filename))
        
    except FileNotFoundError:
        loaded_model = load_keras_model("{}.json".format(filename), "{}.h5".format(filename))
        
    return loaded_model
        
    
loaded_models={}
for i in range(1,11):
    model = load_model("SKU{}_best_model".format(i))
    loaded_models[i] = model

# loaded_models[9].compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
# loaded_models[9].evaluate(X[9], y[9], verbose=0)


In [47]:
# Evaluate loaded model on test data
# loaded_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
# score = loaded_model.evaluate(Xs, y, verbose=0)
# print("%s: %.0f"%  (loaded_model.metrics_names[1], score[1]))
# print("%s: %.1f"%  (loaded_model.metrics_names[2], score[2]/10000000))

# Remember cv is cross validation, which is set to 2 earlier. Thus, X is divided into 2 subsets for train/test process
scores=[]
model_family=[]
for sku in range(1,11):
    
    # Identify the model family
    model = loaded_models[sku]
    if isinstance(model, RandomForestRegressor):
        model_family.append('RF')
    else:
        model_family.append('MLP')

    # Obtain the cross validation score
    score = -cross_val_score(model,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)
    scores.append(score)        

loaded_model_scores = pd.DataFrame({
    'CV Score':scores,
    'Family':model_family
})
loaded_model_scores.index += 1

loaded_model_scores

,CV Score,Family
1,361.0,RF
2,385.0,RF
3,585.0,RF
4,743.0,RF
5,370.0,RF
6,555.0,RF
7,725.0,RF
8,487.0,RF
9,608.0,MLP
10,384.0,RF
